In [15]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
# Reading meta daat downloaded from the website 

df = pd.read_json('Forest_Monitoring/input/Atlas/Atlas_data_frame.json')


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   index                              112 non-null    object 
 1   id                                 112 non-null    int64  
 2   pageProps.projects.hits.hits.sort  112 non-null    int64  
 3   project_id                         112 non-null    int64  
 4   type                               112 non-null    object 
 5   regionShortCode                    111 non-null    object 
 6   developer                          112 non-null    object 
 7   name                               112 non-null    object 
 8   region                             112 non-null    object 
 9   polygonCoordinate                  112 non-null    object 
 10  square                             112 non-null    float64
 11  cidScreenShot                      112 non-null    object 

In [16]:
#Geometry editing
def remove_trailing_zeros(s):
  
    if s.startswith('list(c(') and s.endswith(')'):
       
        s = s[7:-1]
  
    list_of_strings = s.split(',')
   
    list_of_strings = [s.strip().lstrip('c(').rstrip(')') for s in list_of_strings]
    
    list_of_floats = list(map(float, list_of_strings))
   
    while list_of_floats and list_of_floats[-1] == 0.0:
        list_of_floats.pop()
   
    return ', '.join(map(str, list_of_floats))


df['polygonCoordinate'] = df['polygonCoordinate'].apply(remove_trailing_zeros)

In [17]:
import pandas as pd
import simplekml
import os


if not os.path.exists('kml'):
    os.makedirs('kml')

#
updated_polygon_coords = []

for index, row in df.iterrows():
   
    coords = list(map(float, row['polygonCoordinate'].split(',')))
    

    half = len(coords) // 2
    lats = coords[:half]
    longs = coords[half:]
    
 
    polygon_coords = list(zip(lats, longs))
   
    updated_polygon_coords.append(polygon_coords)
    
   
    kml = simplekml.Kml()
    pol = kml.newpolygon(name="Polygon{}".format(index), outerboundaryis=polygon_coords)
    pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.white)
    pol.style.linestyle.color = simplekml.Color.red
    kml.save(os.path.join("kml", "Polygon{}.kml".format(index)))

df['polygonCoordinates'] = updated_polygon_coords

In [18]:
df.to_csv('/home/idisc02/Forest_Monitoring/input/Atlas/OPF.csv', index=False)

In [20]:
#Shape file generation


polygons = [Polygon(polygon_coords) for polygon_coords in updated_polygon_coords]


gdf = gpd.GeoDataFrame(geometry=polygons)


gdf.crs = "EPSG:4326"


gdf.to_file("opf_shapefile.shp")